In [1]:
import pandas as pd
from sqlalchemy import create_engine
pd.options.mode.chained_assignment = None

In [ ]:
csv_file = "Resources/tweets_data.csv"
elon_musk_df = pd.read_csv(csv_file)
elon_musk_df.head()

In [ ]:
elon_musk_tweet_df = elon_musk_df[['date', 'tweet']]
elon_musk_tweet_df.head()

In [2]:
csv_file_2 = "Resources/stocks_data.csv"
tesla_df = pd.read_csv(csv_file_2)
tesla_df.head(2)

,Date,Close/Last,Volume,Open,High,Low
0,7/17/20,"$1,500.84",9329972,"$1,513.45","$1,537.51","$1,490"
1,7/16/20,"$1,500.64",14300790,"$1,477.16","$1,531.71","$1,466"


In [3]:
tesla_df.columns

Index(['Date', ' Close/Last', ' Volume', ' Open', ' High', ' Low'], dtype='object')

In [ ]:
tesla_df.reset_index(drop=True)

In [ ]:
tesla_stock_df = tesla_df[['Date', ' Close/Last', ' Open', ' High', ' Low']]
tesla_stock_df.head()

In [ ]:
tesla_stock_df.rename(columns={'Date':'date', ' Close/Last':'Close', ' Open':'Open', ' High':'High', ' Low':'Low'},inplace=True)
tesla_stock_df.head()

In [ ]:
tesla_stock_df['Close']=tesla_stock_df['Close'].str.lstrip('$').str.rstrip(' ').replace(',','')
tesla_stock_df=tesla_stock_df.replace(',','', regex=True)
tesla_stock_df['Close']=tesla_stock_df['Close'].astype(float)

In [ ]:
tesla_stock_df['Open']=tesla_stock_df['Open'].str.lstrip('$').str.rstrip(' ').replace(',','')
tesla_stock_df=tesla_stock_df.replace(',','', regex=True)
tesla_stock_df['Open']=tesla_stock_df['Open'].astype(float)

In [ ]:
tesla_stock_df['High']=tesla_stock_df['High'].str.lstrip('$').str.rstrip(' ').replace(',','')
tesla_stock_df=tesla_stock_df.replace(',','', regex=True)
tesla_stock_df['High']=tesla_stock_df['High'].astype(float)

In [ ]:
tesla_stock_df['Low']=tesla_stock_df['Low'].str.lstrip('$').str.rstrip(' ')
tesla_stock_df=tesla_stock_df.replace(',','', regex=True)
tesla_stock_df['Low']=tesla_stock_df['Low'].astype(float)

In [ ]:
tesla_stock_df.dtypes

In [ ]:
tesla_stock_df['Open_Close']=tesla_stock_df['Open']-tesla_stock_df['Close']
tesla_stock_df.head()

In [ ]:
tesla_stock_df['High_Low']=tesla_stock_df['High']-tesla_stock_df['Low']
tesla_stock_df.head()

In [ ]:
elon_musk_tweet_df.head()

In [ ]:
elon_tweet_df=elon_musk_tweet_df[elon_musk_tweet_df['tweet'].str.contains("tesla")]
elon_tweet_df

In [ ]:
joined_data_df=pd.merge(elon_tweet_df,tesla_stock_df,how='inner',on='date')
joined_data_df